# Working with MS Word documents
docx is MS WOrd open xml format. The file is essentially a zipped xml file, with text, objects, styles, etc. So in theory, you can open one like this:

`f = open('data/sample_doc.docx', 'r')`

The format started to be used in Word 2007.

### python-docx package
python-docx package seems to be a goto package for working with docx in python. I didn't do a thorough search, but there seems to be few alternatives. So I've installed it and experimeting with it here.

To install, either use pip or for conda:

`conda install -c conda-forge python-docx`

In [2]:
import docx

#### Working with existing docx documents

In [21]:
doc = docx.Document('data/sample_doc.docx')
doc.tables

In [22]:
doc.tables[-1]

In [23]:
doc.core_properties.title

''

In [24]:
for i, row in enumerate(doc.tables[-1].rows):
    print()
    for cell in row.cells:
        print("{}".format(cell.text), end='|')


Group2|Group2|Group2|Group2|Group2|
Pest Name|Present |Interesting|Fluffy|Information|
Name 3|Y|N|N|bla: lalalalala lal alalala alala alamala|
Name 4|Y|N|N|bla: Some description text|
Name 5|N|N|N|bla: Some text that is not interesting and not intended to be read by a sensible being)
la: Some text that is not interesting and not intended to be read by a sensible being|
Name 6|Y|N|N|bla: Some text that is not interesting and not intended to be read by a sensible being|

In [25]:
# Read a table in the docx document into a dataframe
import pandas as pd

def load_table(docx_table):
    table_data = []
    
    for i, row in enumerate(docx_table.rows):
        row_data = []
        for cell in row.cells:
            row_data.append(cell.text)
        table_data.append(row_data)

    return pd.DataFrame(table_data)
    

In [26]:
df = load_table(doc.tables[-1])
df

,0,1,2,3,4
0,Group2,Group2,Group2,Group2,Group2
1,Pest Name,Present,Interesting,Fluffy,Information
2,Name 3,Y,N,N,bla: lalalalala lal alalala alala alamala
3,Name 4,Y,N,N,bla: Some description text
4,Name 5,N,N,N,bla: Some text that is not interesting and not...
5,Name 6,Y,N,N,bla: Some text that is not interesting and not...


In [28]:
df = load_table(doc.tables[3])
df

,0,1,2,3,4
0,Group1,Group1,Group1,Group1,Group1
1,Name,Present,Interesting,Fluffy,Information
2,Name 1,Y,N,N,Some info about name 1
3,Name 2,Y,N,N,bla: notinteresting
